In [1]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

2024-05-20 18:50:42.926227: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-20 18:50:42.926283: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-20 18:50:42.927799: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# load data frames
df_train = pd.read_csv('/kaggle/input/dataframes/train.csv')
df_val = pd.read_csv('/kaggle/input/dataframes/val.csv')
df_test = pd.read_csv('/kaggle/input/dataframes/val.csv')

# load visual embeddings
with open('/kaggle/input/v-embeds/v_embeds_train.pkl', 'rb') as f:
    v_embeds_train = pickle.load(f) 
    
with open('/kaggle/input/v-embeds/v_embeds_val.pkl', 'rb') as f:
    v_embeds_val = pickle.load(f) 
    
with open('/kaggle/input/v-embeds/v_embeds_test.pkl', 'rb') as f:
    v_embeds_test = pickle.load(f) 

In [3]:
print(len(v_embeds_train))
print(v_embeds_train[0].shape)
stacked_tensor = torch.stack(v_embeds_train, dim=0)

print(stacked_tensor.shape)

1962
torch.Size([100, 1024])
torch.Size([1962, 100, 1024])


In [4]:
# stack visual features 
X_train = torch.stack(v_embeds_train, dim=0).cpu().numpy()
X_val = torch.stack(v_embeds_val, dim=0).cpu().numpy()
X_test = torch.stack(v_embeds_test, dim=0).cpu().numpy()
print(X_train.shape, X_val.shape, X_test.shape)

(1962, 100, 1024) (109, 100, 1024) (109, 100, 1024)


In [5]:
# convert labels to numerical values
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train['AUTHOR']) # RICHTIG SO??? fit und dann transform?
y_val = label_encoder.transform(df_val['AUTHOR'])
y_test = label_encoder.transform(df_test['AUTHOR'])

In [9]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(100, 1024)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')


#test_predictions = model.predict(X_test)
#model.save("baseline_model.h5")

Epoch 1/10
17/62 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1444 - loss: 39.5429

W0000 00:00:1716231244.287808    1006 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.2032 - loss: 28.1259

W0000 00:00:1716231245.960098    1009 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.2041 - loss: 27.9419 - val_accuracy: 0.2294 - val_loss: 7.0492
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5638 - loss: 2.5657 - val_accuracy: 0.3211 - val_loss: 6.5380
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7959 - loss: 0.8556 - val_accuracy: 0.3028 - val_loss: 5.9349
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9142 - loss: 0.2692 - val_accuracy: 0.2936 - val_loss: 5.0655
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9745 - loss: 0.0730 - val_accuracy: 0.3578 - val_loss: 5.6620
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9942 - loss: 0.0252 - val_accuracy: 0.3303 - val_loss: 4.9072
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 0.0027 - val_accuracy: 0.3303 - val_loss: 4.7305
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 0.3303 - val_loss: 4